# LSH Implementation for Football Player Scouting

This workflow implements a Locality Sensitive Hashing (LSH) system to find similar players based on performance statistics. It uses P-Stable Distributions to approximate Euclidean Distance, combined with a Clustering post-filter to ensure tactical relevance.

## Libraries and Data Loading

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np

In [2]:
df = pd.read_csv("../../data/processed/players_w_clusters.csv")

In [3]:
df.head()

,player,team,nation,pos,age,90s Playing Time,npxG Per 90 Minutes,G-PK Per 90 Minutes,xAG Per 90 Minutes,xG+xAG Per 90 Minutes,...,PrgR Receiving,TklW Tackles,Int,Won Aerial Duels,Fls Performance,Fld Performance,Att 3rd Touches,Mid 3rd Touches,Def 3rd Touches,cluster
0,Aaron Cresswell,West Ham,ENG,"DF,FW",33.0,4.8,0.00,0.0,0.09,0.09,...,6,2,3,6,2,3,61,141,125,0
1,Aaron Hickey,Brentford,SCO,DF,21.0,7.9,0.03,0.0,0.01,0.04,...,13,9,3,1,10,16,76,155,132,0
2,Aaron Malouda,Lille,FRA,FW,17.0,0.0,0.00,0.0,0.00,0.00,...,0,0,0,0,0,0,0,0,0,0
3,Aaron Ramsdale,Arsenal,ENG,GK,25.0,6.0,0.00,0.0,0.00,0.00,...,0,0,0,0,0,1,0,11,186,0
4,Aaron Ramsey,Burnley,ENG,"MF,FW",20.0,5.9,0.06,0.0,0.07,0.13,...,17,14,1,4,7,3,65,102,63,0


In [4]:
df.columns

Index(['player', 'team', 'nation', 'pos', 'age', '90s Playing Time',
       'npxG Per 90 Minutes', 'G-PK Per 90 Minutes', 'xAG Per 90 Minutes',
       'xG+xAG Per 90 Minutes', 'Sh/90 Standard', 'G/Sh Standard', 'PrgP',
       'KP', 'PPA', 'CrsPA', 'PrgC Carries', 'CPA Carries', 'Succ Take-Ons',
       'PrgR Receiving', 'TklW Tackles', 'Int', 'Won Aerial Duels',
       'Fls Performance', 'Fld Performance', 'Att 3rd Touches',
       'Mid 3rd Touches', 'Def 3rd Touches', 'cluster'],
      dtype='object')

## Feature Engineering

Raw "Total" statistics are biased towards players with more minutes. To make a fair comparison , we convert volume metrics into "Per 90 Minutes" stats.


In [5]:

# Metadata
cols_meta = ['player', 'team', 'nation', 'pos', 'age', 'cluster', '90s Playing Time']

# Volume Metrics must be divided by '90s Playing Time'
cols_to_per90 = [
    'PrgP', 'KP', 'PPA', 'CrsPA', 'PrgC Carries', 'CPA Carries', 
    'Succ Take-Ons', 'PrgR Receiving', 'TklW Tackles', 'Int', 
    'Won Aerial Duels', 'Fls Performance', 'Fld Performance', 
    'Att 3rd Touches', 'Mid 3rd Touches', 'Def 3rd Touches'
]

# ready Metrics 
cols_ready = [
    'npxG Per 90 Minutes', 'G-PK Per 90 Minutes', 'xAG Per 90 Minutes',
    'Sh/90 Standard', 'G/Sh Standard'
]


feature_names = []

for col in cols_ready:
    if col in df.columns:
        feature_names.append(col)

for col in cols_to_per90:
    if col in df.columns:
        new_col = col + '_p90'
        # Direct division
        df[new_col] = df[col] / df['90s Playing Time']
        feature_names.append(new_col)

## Data Cleaning and Standardization

In [6]:
# selecting the features
X_df = df[feature_names]

# important! replacing infinity with NaN
# handles cases where division by zero occurred
X_df = X_df.replace([np.inf, -np.inf], np.nan)

# fill NaNs with 0
X = X_df.fillna(0).values

# apply Standard Scaler
print(f"Features ready for scaling: {X.shape[1]}")
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

Features ready for scaling: 21


## LSH Engine (Euclidean Distance)

This class implements the Random Projection method.

- Projections (a): Random vectors drawn from a Gaussian distribution. They simulate "slicing" the multidimensional space.

- Bucket Width (W): Determines how wide a "slice" is. A larger W makes the search more tolerant (finds more neighbors), while a smaller W is stricter.

- Index: Maps every player's vector to a specific "Bucket ID" (Signature).

In [7]:
class LSH_Euclidean_Manual:
    def __init__(self, num_projections=10, bucket_width=4.0, input_dim=None, seed=42):
        """
        Implementation of LSH for P-Stable Distributions (Euclidean Distance).
        
        Args:
            num_projections (k): Length of the hash signature. 
                                 Higher = Stricter, more precise search.
            bucket_width (W):    The width of the projection "bucket".
                                 Higher = More tolerant (finds broader matches).
                                 Lower = Very strict (might return 0 results).
            input_dim:           Number of features in the data.
        """
        np.random.seed(seed)
        self.W = bucket_width
        self.k = num_projections
        
        # create Random Hyperplanes
        # Shape: (k, input_dim)
        # this projects the multi-dimensional player vector onto a random line.
        self.projections = np.random.randn(self.k, input_dim)
        
        # 2. Create Random Bias (Uniform Distribution between 0 and W)
        # Shape: (k,)
        # This shifts the grid slightly to handle edge cases.
        self.bias = np.random.uniform(0, self.W, size=self.k)
        
        # Hash Table to store buckets: { "hash_signature": [player_indices] }
        self.hash_table = {}

    def _compute_hash(self, vector):
        """ Applies the LSH formula: floor( (a . v + b) / W ) """
        
        # Dot product (Projection)
        dot_product = np.dot(self.projections, vector)
        
        # Add bias, divide by width, and floor to nearest integer
        hash_vector = np.floor((dot_product + self.bias) / self.W).astype(int)
        
        # Convert numpy array to tuple (to be used as dictionary key)
        return tuple(hash_vector)

    def index(self, data_matrix):
        """ Hashes the entire database into buckets """
        self.hash_table = {} # Reset
        
        for idx, vector in enumerate(data_matrix):
            signature = self._compute_hash(vector)
            
            if signature not in self.hash_table:
                self.hash_table[signature] = []
            self.hash_table[signature].append(idx)
            
        print(f"Indexing complete. Created {len(self.hash_table)} unique buckets.")

    def query(self, vector):
        """ Finds players falling into the exact same bucket as the input vector """
        signature = self._compute_hash(vector)
        
        # Retrieve neighbors (or empty list if no one else is there)
        candidates_indices = self.hash_table.get(signature, [])
        return candidates_indices, signature

## Configuration and Indexing

In [8]:
# data references to make the variables more interpretable
df_players = df
data_matrix = X_scaled
feat_names = feature_names


# configure LSH
input_dims = data_matrix.shape[1]


lsh = LSH_Euclidean_Manual(num_projections=10, bucket_width=7.5, input_dim=input_dims)

# index data
lsh.index(data_matrix)

# Verification of parameters
print(f"K: {lsh.k}, W: {lsh.W}")

Indexing complete. Created 644 unique buckets.
K: 10, W: 7.5


## Execution

In [ ]:
search_query = "Virgil van Dijk" 

# player index
matches = df_players[df_players['player'].str.contains(search_query, case=False, na=False)]

if len(matches) == 0:
    print(f"Error: Player '{search_query}' not found in the dataset.")
else:
    # Target Info from the first match
    target_idx = matches.index[0]
    target_player_name = matches.iloc[0]['player']
    target_vector = data_matrix[target_idx]      # n vector of the target
    target_cluster = matches.iloc[0]['cluster']
    target_pos = matches.iloc[0]['pos']

    print(f"\n  Searching for players similar to: {target_player_name}")
    print(f"   Position: {target_pos} | Cluster: {target_cluster}")
    print("-" * 60)

    # querying THE LSH ENGINE
    neighbor_indices, _ = lsh.query(target_vector)
    
    # Filter out the player themselves
    neighbor_indices = [idx for idx in neighbor_indices if idx != target_idx]

    # create a copy of the candidate rows
    candidates = df_players.iloc[neighbor_indices].copy()
    
    # Keep only those in the same cluster
    final_results = candidates[candidates['cluster'] == target_cluster].copy()

    if len(final_results) == 0:
        print(f" LSH found candidates, but none matched Cluster {target_cluster}.")
    else:
        # similarity metric (Euclidean Distance)
        distances = []
        
        # loop through the final candidates to calculate exact distance
        for idx in final_results.index:
            candidate_vector = data_matrix[idx]
            
            # Euclidean Distance: sqrt(sum((a-b)^2))
            # 0.0 means identical, tthe lower the better.
            dist = np.linalg.norm(target_vector - candidate_vector)
            distances.append(dist)
        
        # distance column to the dataframe
        final_results['Distance'] = distances
        
        # srting
        final_results = final_results.sort_values('Distance', ascending=True)

        print(f"  Found {len(final_results)} matches. Sorted by similarity (Distance):")
        print("(Lower is more similar)\n")
        
        # display specific columns including Position ('pos')
        cols_to_show = ['player', 'team', 'pos', 'age', 'Distance']
        print(final_results[cols_to_show].head(10))


  Searching for players similar to: Virgil van Dijk
   Position: DF | Cluster: 4
------------------------------------------------------------
  Found 75 matches. Sorted by similarity (Distance):
(Lower is more similar)

               player           team pos   age  Distance
2318      Willi Orban     RB Leipzig  DF  30.0  1.022670
2000    Ronald Araújo      Barcelona  DF  24.0  1.246347
889   Ibrahima Konaté      Liverpool  DF  24.0  1.361724
901     Igor Zubeldia  Real Sociedad  DF  26.0  1.492260
1661  Mohammed Salisu         Monaco  DF  24.0  1.504583
2355    Yeray Álvarez  Athletic Club  DF  28.0  1.726404
384    Cedric Zesiger      Wolfsburg  DF  25.0  1.737949
1330      Logan Costa       Toulouse  DF  22.0  1.763406
1737      Nico Elvedi       Gladbach  DF  26.0  1.844493
272        Bamo Meïté        Lorient  DF  21.0  1.887647
